In [4]:
import spacy

nlp = spacy.load("es_core_news_lg")

doc = nlp("corren")
token = doc[0]

def change_morph_feature(token, feature, new_value):
   morph = token.morph.to_dict()
   morph[feature] = new_value
   morph_string = '|'.join(f'{k}={v}' for k, v in morph.items())
   new_form = token.doc.vocab.morphology.assign_tag(token.lemma_, token.pos_, morph_string)
   return new_form

# Print original morphology
print(f"Original word: {token.text}")
print(f"Original morphology: {token.morph}")

# Change number from Plur to Sing
new_word = change_morph_feature(token, "Number", "Sing")

print(f"\nNew word: {new_word}")

Original word: corren
Original morphology: Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin


AttributeError: 'spacy.morphology.Morphology' object has no attribute 'assign_tag'

In [6]:
import spacy
from spacy.symbols import VERB

nlp = spacy.load("es_core_news_lg")
doc = nlp("corren")
token = doc[0]

# Print original info
print(f"Original: {token.text} {token.morph}")

# Get inflection table for this lemma
verb_forms = token.doc.vocab.morphology.lemmatizer.lookup_table.get((VERB, token.lemma_))
if verb_forms:
    # Find form with same features but different number
    morph = token.morph.to_dict()
    morph["Number"] = "Sing"  # Change desired feature
    target_morph = '|'.join(f'{k}={v}' for k, v in morph.items())
    
    for form in verb_forms:
        if token.doc.vocab.strings[form[1]] == target_morph:
            new_word = token.doc.vocab.strings[form[0]]
            print(f"New: {new_word} {target_morph}")
            break

Original: corren Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin


AttributeError: 'spacy.morphology.Morphology' object has no attribute 'lemmatizer'

In [7]:
import spacy

nlp = spacy.load("es_core_news_lg")
doc = nlp("corren")
token = doc[0]

# Get morphologizer
morphologizer = nlp.get_pipe("morphologizer")

# Create new doc with lemma
new_doc = nlp(token.lemma_)
new_token = new_doc[0]

# Set target morphology
morph = token.morph.to_dict()
morph["Number"] = "Sing"

# Apply morphology
morphologizer.set_annotations([new_token], {"morph": [morph]})

print(f"Original: {token.text} {token.morph}")
print(f"New: {new_token.text} {new_token.morph}")

TypeError: Cannot convert spacy.tokens.token.Token to spacy.tokens.doc.Doc

In [10]:
import spacy
from mlconjug3 import Conjugator

nlp = spacy.load("es_core_news_lg")
conjugator = Conjugator(language='es')

def change_verb_number(verb_token):
   morph = verb_token.morph.to_dict()
   verb = conjugator.conjugate(verb_token.lemma_)
   
   # Map spaCy mood/tense to mlconjug3 format
   mood_map = {"Ind": "Indicativo", "Sub": "Subjuntivo", "Imp": "Imperativo"}
   tense_map = {"Pres": "Presente", "Past": "Pretérito", "Imp": "Imperfecto"}
   
   mood = mood_map[morph.get("Mood", "Ind")]
   tense = tense_map[morph.get("Tense", "Pres")]
   person = morph.get("Person", "3")
   
   tense_forms = verb.conjug_info[mood][tense]
   
   if morph.get("Number") == "Plur":
       new_form = tense_forms[f"{person}s"]
   else:
       new_form = tense_forms[f"{person}p"]
       
   return new_form

# Test
doc = nlp("corren")
token = doc[0]

print(f"Original: {token.text} {token.morph}")
new_form = change_verb_number(token)
print(f"New form: {new_form}")

Original: corren Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin


KeyError: 'Presente'

In [16]:
import spacy
from mlconjug3 import Conjugator

nlp = spacy.load("es_core_news_lg")
conjugator = Conjugator(language='es')

def change_verb_number(verb_token):
   morph = verb_token.morph.to_dict()
   verb = conjugator.conjugate(verb_token.lemma_)
   
   # Print conjugation table to see actual structure
   #print("Conjugation info:")
   #for mood, tenses in verb.conjug_info.items():
       #print(f"\n{mood}:")
       #for tense, forms in tenses.items():
           #print(f"  {tense}: {forms}")

   # For debugging
   #print(f"\nMorphology: {morph}")
   
   return verb  # Temporary return for debugging

# Test
doc = nlp("corren")
token = doc[0]

print(f"Original: {token.text} {token.morph}")
change_verb_number(token)

Original: corren Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin


mlconjug3.verbs.verbs.VerbEs(correr)

In [15]:
change_verb_number(token)